In [29]:
from os import path
import os

import astropy.coordinates as coord
from astropy.constants import G
from astropy.io import fits
from astropy.table import Table, QTable, join
from astropy.time import Time
import astropy.units as u
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tqdm import tqdm
import requests
import yaml

from hq.config import HQ_CACHE_PATH, config_to_alldata, config_to_jokerparams
from hq.plot import plot_two_panel, plot_phase_fold
from hq.data import get_rvdata
from hq.physics_helpers import period_at_surface, stellar_radius
from hq.mass_helpers import get_martig_mass

from thejoker.plot import plot_rv_curves
from thejoker import JokerSamples, JokerParams, TheJoker

from twobody.transforms import get_m2_min

In [85]:
plot_path = path.expanduser('~/projects/bh/plots/')
data_path = path.expanduser('~/projects/bh/data/')

In [3]:
name = 'dr16'

In [4]:
joker_f = h5py.File('../cache/{0}/thejoker-samples.hdf5'.format(name), 'r')
emcee_f = h5py.File('../cache/{0}/emcee-samples.hdf5'.format(name), 'r')

In [5]:
with open("../cache/{}/config.yml".format(name), "r") as f:
    config = yaml.load(f.read())
    
allstar, allvisit = config_to_alldata(config)
poly_trend = config['hyperparams']['poly_trend']

In [6]:
master = QTable.read('../cache/{0}/metadata-master.fits'.format(name))
master = join(master, allstar, keys='APOGEE_ID')

In [7]:
mass_tbl = Table.read('../data/shite_m1_estimates.fits')
_, idx = np.unique(mass_tbl['APOGEE_ID'], return_index=True)
mass_tbl = mass_tbl[idx]
master = join(master, mass_tbl, keys='APOGEE_ID')

In [8]:
gaia = Table.read('../data/allStar-r12-gaiadr2.fits')
_, idx = np.unique(gaia['apogee_id'], return_index=True)
gaia = gaia[idx]
gaia.rename_column('apogee_id', 'APOGEE_ID')
master = join(master, gaia, keys='APOGEE_ID')

In [9]:
mask = (((master['STARFLAG'] & np.sum(2**np.array([2, 9]))) == 0) & # BRIGHT_NEIGHBOR, PERSIST_HIGH
        ((master['ASPCAPFLAG'] & np.sum(2**np.array([7]))) == 0)) # STAR_WARN
t = master[mask]
len(t)

154911

In [10]:
llr = t['max_unmarginalized_ln_likelihood'] - t['robust_constant_ln_likelihood']

In [11]:
llr_mask = (llr > 4) & (t['max_unmarginalized_ln_likelihood'] > 0) & (np.isfinite(t['m1']))
llr_mask.sum()

8477

In [12]:
def m_f(P, K, e):
    """Binary mass function."""
    mf_circ = P * K**3 / (2*np.pi * G)
    return mf_circ.to(u.Msun) * (1 - e**2)**1.5

## Select the candidates

In [60]:
cool_ids = []
for row in tqdm(t[llr_mask]):
    apogee_id = row['APOGEE_ID']
    m1 = row['m1'] * u.Msun
    
    if apogee_id in emcee_f.keys():
        samples = JokerSamples.from_hdf5(emcee_f[apogee_id], 
                                         poly_trend=poly_trend)
    else:
        samples = JokerSamples.from_hdf5(joker_f[apogee_id], 
                                         poly_trend=poly_trend)
    
    # data = get_rvdata(allvisit[allvisit['APOGEE_ID'] == apogee_id])
    # _ = plot_rv_curves(samples=samples, data=data)
    
    m2_min = get_m2_min(np.repeat(m1, len(samples)),
                        m_f(samples['P'], samples['K'], samples['e']))
    
    if np.nanpercentile(m2_min.value, 15) * u.Msun > m1 and 0.2 < m1.value < 2.5:
        cool_ids.append(apogee_id)

In [13]:
# cool_ids = ['2M00163253+8556588', '2M00251183+0137366', '2M00385903-0109499', '2M01415794+8520066', '2M02484723+0719196', '2M03135991+3831519', '2M03454440+2413132', '2M04015112+5316431', '2M04405488+4746463', '2M06185816+4002167', '2M06501711-0547415', '2M08001560+2550021', '2M08024840+4129341', '2M08213897+4559430', '2M08231995-3657095', '2M08511868+1147026', '2M14332948+0958420', '2M14544500+4626456', '2M15074403+3218142', '2M15091513+3143072', '2M15551237+2629107', '2M17021700-2145531', '2M19340471+2632110', '2M20593168+5559227', '2M22001521+5559468']

In [14]:
print(cool_ids)

['2M00163253+8556588', '2M00251183+0137366', '2M00385903-0109499', '2M01415794+8520066', '2M02484723+0719196', '2M03135991+3831519', '2M03454440+2413132', '2M04015112+5316431', '2M04405488+4746463', '2M06185816+4002167', '2M06501711-0547415', '2M08001560+2550021', '2M08024840+4129341', '2M08213897+4559430', '2M08231995-3657095', '2M08511868+1147026', '2M14332948+0958420', '2M14544500+4626456', '2M15074403+3218142', '2M15091513+3143072', '2M15551237+2629107', '2M17021700-2145531', '2M19340471+2632110', '2M20593168+5559227', '2M22001521+5559468']


In [15]:
len(cool_ids)

25

In [89]:
candidates = master[np.isin(master['APOGEE_ID'], cool_ids)]
assert len(candidates) == len(cool_ids)
Table(candidates).write('../../bh/data/candidates.fits', overwrite=True)